In [0]:
#Importing surprise packages
from surprise import BaselineOnly
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import cross_validate
import pandas as pd
import numpy as np
import os, io
from surprise import SVD
from surprise import accuracy

In [0]:
file_path = os.path.expanduser('train.csv') 
reader = Reader(sep=',',rating_scale=(0,100),skip_lines=1)
ldata = Dataset.load_from_file(file_path, reader=reader)  #reading data from file to surprise datastructure dataset
trainingSet = ldata.build_full_trainset()                 #building training set from raw data
testSet = trainingSet.build_anti_testset(fill=None)       #creating test set of all unknown ratings without imputing with anything.

In [0]:
algo = SVD('n_epochs'= 10,'lr_all'= 0.005,'reg_all'= 0.02,'n_factors'= 150)  #SVD model with tuned parameters                                            #
algo.fit(trainingSet)                                                       #training with SVD on training set
predictions = algo.test(testSet)                                            #prediction(testing) with test set(anti test set)

In [0]:
df = pd.DataFrame(np.array(predictions).reshape(-1,5), columns = list("abcde"))  #converting prediction(prediction datastructure of surprise) to pandas dataframe
df=df.drop('c', 1)  #dropping unwanted columns                                                              
df=df.drop('e',1)
df['d']=df['d'].astype(int)  #converting to int-rounding off ratings
df2=pd.read_csv('test.csv')  #reading test file
df2['Final'] = df2.User.astype(str).str.cat(df2.Track.astype(str), sep='-')  #merging user and track as required in output file
df['Final1'] = df.a.astype(str).str.cat(df.b.astype(str), sep='-')         
dft = df[df['Final1'].isin(df2['Final'])]  #comparing test dataframe and prediction dataframe to extract ratings of test file
df12=dft.drop('a',1)  #droping unwanted columns
df123=df12.drop('b',1)
df6=df123.rename(index=str, columns={"d": "Rating", "Final1": "Id"})  #renaming columns as required in test file
dff = df6[['Id', 'Rating']]  #creating new dataframe similar to required output file
dff.to_csv("out.csv", index=False)  #converting dataframe to csv